In [8]:
# setting the N value

N = 5;

# points in t direction
# trigpts returns N equally spaced points on an interval -1 to 1
# range is kind of like linspace! the length

t = pi* range(-1,1, length=N+1)[1:end-1];
t = reshape(t,:,1)


# chebpts returns chebyshev points of the second kind
# generates N+1 cheb pts in the interval [-1, 1], so includes endpoints
# points are clustered near the endpoints

r = [cos((pi*k)/(N-1)) for k in (N-1):-1:0];
z = [cos((pi*k)/(N-1)) for k in (N-1):-1:0];


# going from N-1 to 0 ensures that endpoints are included bc when you 
#plug in (N-1) and 0 into the formula, you get the endpoints included -1 and 1 regardless of N
# cosine part is just a formula for cheb pts of second kind

display(t)
display(z)
display(r)

5×1 reshape(::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, 5, 1) with eltype Float64:
 -3.141592653589793
 -1.8849555921538759
 -0.6283185307179586
  0.6283185307179586
  1.8849555921538759

5-element Vector{Float64}:
 -1.0
 -0.7071067811865475
  6.123233995736766e-17
  0.7071067811865476
  1.0

5-element Vector{Float64}:
 -1.0
 -0.7071067811865475
  6.123233995736766e-17
  0.7071067811865476
  1.0

In [9]:
# now to make the grid! this was a bit incorrect because we need the different layers? 
#I found a package that works better!

# tt = 
rr = hcat([r for _ in 1:N]...)
zz = vcat([transpose(z) for _ in 1:N]...)
 
display(rr)
display(zz)

5×5 Matrix{Float64}:
 -1.0          -1.0          -1.0          -1.0          -1.0
 -0.707107     -0.707107     -0.707107     -0.707107     -0.707107
  6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17
  0.707107      0.707107      0.707107      0.707107      0.707107
  1.0           1.0           1.0           1.0           1.0

5×5 Matrix{Float64}:
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

In [10]:
# same row vector vertical stacked
rr = [ri for ri in r, zi in z, ti in t]

# same column vector horizontal stacked
zz = [zi for ri in r, zi in z, ti in t]

# every entry in a matrix the same, but each layer has different value
tt = [ti for ri in r, zi in z, ti in t]

display(rr)
display(zz)
display(tt)


5×5×5×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -1.0          -1.0          -1.0          -1.0          -1.0
 -0.707107     -0.707107     -0.707107     -0.707107     -0.707107
  6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17
  0.707107      0.707107      0.707107      0.707107      0.707107
  1.0           1.0           1.0           1.0           1.0

[:, :, 2, 1] =
 -1.0          -1.0          -1.0          -1.0          -1.0
 -0.707107     -0.707107     -0.707107     -0.707107     -0.707107
  6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17
  0.707107      0.707107      0.707107      0.707107      0.707107
  1.0           1.0           1.0           1.0           1.0

[:, :, 3, 1] =
 -1.0          -1.0          -1.0          -1.0          -1.0
 -0.707107     -0.707107     -0.707107     -0.707107     -0.707107
  6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17   6.12323e-17
  0.707107      0.707107      0.707107      0.707107      0.707107
  1.

5×5×5×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 2, 1] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 3, 1] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 4, 1] =
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0
 -1.0  -0.707107  6.12323e-17  0.707107  1.0

[:, :, 5,

5×5×5×1 Array{Float64, 4}:
[:, :, 1, 1] =
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159
 -3.14159  -3.14159  -3.14159  -3.14159  -3.14159

[:, :, 2, 1] =
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496
 -1.88496  -1.88496  -1.88496  -1.88496  -1.88496

[:, :, 3, 1] =
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319
 -0.628319  -0.628319  -0.628319  -0.628319  -0.628319

[:, :, 4, 1] =
 0.628319  0.628319  0.628319  0.628319  0.628319
 0.628319  0.628319  0.628319  0.628319  0.628319
 0.628319  0.628319  0.628319  0.62

In [11]:
# get the matrices we want: C02, D2, A, coF
using LinearAlgebra, SparseArrays

# C01: Convert Chebyshev-T (T) coefficients to Chebyshev-U (U)
C01 = spdiagm(0 => 0.5*ones(N), 2 => -0.5*ones(N-2))
C01[1,1] = 1.0
println("C01:")
display(C01)
# C12: Convert Chebyshev-U to C^2, the continuous function space
K_1 = 1.0 ./ (1:N)
K_2 = 1.0 ./ (3:N)
C12 = spdiagm(0 => K_1, 2 => -K_2)
println("C12:")
display(C12)
# C02: Convert Chebyshev-T to C^2 (It's C01 * C12)
C02 = C01 * C12
println("C02:")
display(C02)

# D1: Differentiate Chebyshev-T and convert to Chebyshev-U
D1 = spdiagm(1 => 1:N-1)
println("D1: ")
display(D1)
# D2: Differentiate Chebyshev-T twice and convert to C²
D2 = spdiagm(2 => 2 .* (2:N-1))
println("D2: ")
display(D2)
# D0: Differentiate Chebyshev-T once
D0 = C01 \ D1
println("D0: ")
display(D0)

# R0: Multiply Cheb-T by x
R0 = spdiagm(-1 => 0.5*ones(N-1), 1 => 0.5*ones(N-1))
R0[2,1] = 1.0
println("R0:")
display(R0)
# R2: Multiply in C² space
denom = 4:2:2*N+2
K_sub  = ((1:N) ./ denom)[1:N-1]
K_super = ((3:N+2) ./ denom[1:N])[2:N]
R2 = spdiagm(-1 => K_sub, 1 => K_super)
println("R2:")
display(R2)
# R1: Multiply in Cheb-U basis
R1 = C12 \ R2 * C12
println("R1:")
display(R1)

# A: Laplacian operator
A = R2 * R2 * D2 + R2 * C12 * D1
println("A:")
display(A)

# coF: Laplacian multiplier
coF = R2 * R2 * C02
println("coF:")
display(coF)

C01:


5×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 1.0   ⋅   -0.5    ⋅     ⋅ 
  ⋅   0.5    ⋅   -0.5    ⋅ 
  ⋅    ⋅    0.5    ⋅   -0.5
  ⋅    ⋅     ⋅    0.5    ⋅ 
  ⋅    ⋅     ⋅     ⋅    0.5

C12:


5×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 1.0   ⋅   -0.333333    ⋅      ⋅ 
  ⋅   0.5    ⋅        -0.25    ⋅ 
  ⋅    ⋅    0.333333    ⋅    -0.2
  ⋅    ⋅     ⋅         0.25    ⋅ 
  ⋅    ⋅     ⋅          ⋅     0.2

C02:


...

5×5 Matrix{Float64}:
 0.0  0.5  0.0  0.0   0.0
 0.5  0.0  0.5  0.0  -0.166667
 0.0  0.5  0.0  0.5   0.0
 0.0  0.0  0.5  0.0   0.166667
 0.0  0.0  0.0  0.5   0.0

A:


5×5 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
  ⋅    ⋅    1.33333    ⋅     2.66667
  ⋅   0.25   ⋅        2.625   ⋅ 
  ⋅    ⋅    0.666667   ⋅     3.73333
  ⋅    ⋅     ⋅        1.125   ⋅ 
  ⋅    ⋅     ⋅         ⋅     1.6

coF:


5×5 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
 0.166667    ⋅       -0.0138889    ⋅         -0.0666667
  ⋅         0.09375    ⋅         -0.046875     ⋅ 
 0.0833333   ⋅        0.0305556    ⋅         -0.0433333
  ⋅         0.03125    ⋅          0.0260417    ⋅ 
  ⋅          ⋅        0.025        ⋅         -0.00666667

In [23]:
# dummy test function
finit(r, z, θ, t) = r .* sin.(θ) .+ z .* cos.(θ)

# initialize CFS array to hold function values at t = 0
CFS = zeros(size(rr))

# Evaluate initial condition on the full (r, z, θ) grid
for j in 1:N
    CFS[:, j, :] = finit(rr[:,j,:],zz[:,j,:], tt[:,j,:], 0)
end

display(CFS);

5×5×5×1 Array{Float64, 4}:
[:, :, 1, 1] =
 1.0  0.707107   6.12323e-17  -0.707107  -1.0
 1.0  0.707107   2.53633e-17  -0.707107  -1.0
 1.0  0.707107  -6.12323e-17  -0.707107  -1.0
 1.0  0.707107  -1.47828e-16  -0.707107  -1.0
 1.0  0.707107  -1.83697e-16  -0.707107  -1.0

[:, :, 2, 1] =
  1.26007    1.16956    0.951057      0.732549   0.64204
  0.981516   0.891007   0.672499      0.45399    0.363482
  0.309017   0.218508  -7.71572e-17  -0.218508  -0.309017
 -0.363482  -0.45399   -0.672499     -0.891007  -0.981516
 -0.64204   -0.732549  -0.951057     -1.16956   -1.26007

[:, :, 3, 1] =
 -0.221232   0.0157238   0.587785      1.15985    1.3968
 -0.39339   -0.156434    0.415627      0.987688   1.22464
 -0.809017  -0.572061    1.35465e-17   0.572061   0.809017
 -1.22464   -0.987688   -0.415627      0.156434   0.39339
 -1.3968    -1.15985    -0.587785     -0.0157238  0.221232

[:, :, 4, 1] =
 -1.3968    -1.15985    -0.587785     -0.0157238  0.221232
 -1.22464   -0.987688   -0.415627      0.1

In [7]:
# replicate V2C_cyl function
using FFTW
using LinearAlgebra

function V2C_cyl(vals::Array, a::AbstractString)
    # julia uses mapslices with dims=d, which works directly on a specific dimension so no need to manually reshape like in MATLAB

    dim = ndims(vals)
    sz = size(vals)
    
    # check that length(a) = # of dimensions
    @assert length(a) == dim "Length of 'a' must match number of dimensions in vals."

    # loop through dims, deciding which transform to apply
    for d in 1:dim
        # if t, FFT along dim
        if a[d] == 't' || a[d] == 'T'
            vals = mapslices(x -> fft(x) / length(x), vals; dims=d)
        elseif a[d] == '0'
            # do nothing
            continue
        # otherwise dct-ii along dim
        else
            vals = mapslices(x -> dct(x, 2), vals; dims=d)
        end

        # julia uses mapslices, which works directly on a specific dimension so no need to reshape and permute dimensions like in MATLAB
    end

    return vals
end

LoadError: ArgumentError: Package FFTW not found in current path.
- Run `import Pkg; Pkg.add("FFTW")` to install the FFTW package.